In [ ]:
# !pip install selenium
# !pip install bs4
# !pip install beautifulsoup4
# !pip install chromedriver-autoinstaller
# pip install mysql-connector-python
# !pip install openpyxl

In [23]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from requests import get
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
from selenium import webdriver
import chromedriver_autoinstaller
import os

chromedriver_autoinstaller.install()  

In [3]:
def getBrand(url):

    
    url2=url.split('_')
    url3=url2[-1].split('-')
    if len(url3)==2:
        return url3[-1].capitalize()
    else:
        return url2[-1].capitalize()

def getCategory(text):
    
    pattern = r'^([^_]+)'
    match = re.match(pattern, text)

    if match:
        result = match.group(1)
        #print(result)
        res=result.split('-')
        if(len(res)==3):
            resa=res[-2] + " " + res[-1]
            return resa.capitalize()
        elif(len(res)==2):
            resa=res[-1]
            return resa.capitalize()
    else:
        print("No match found.")
        
def modify_list(input_list):
    modified_list = [item.lower().replace(' ', '-').replace('-mobile', '') for item in input_list if 'other' not in item.lower()]
    return modified_list

def returnLink(url,cars,total_count,page):
    if 'car' in url:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        html = driver.page_source
        parsed_html = BeautifulSoup(html, "html.parser")
        containers = parsed_html.find("ul", {"class" : "ba608fb8 de8df3a3"})
        driver.close()
        try:
            for item in containers:
                return url
        except TypeError:
            
            url=f'https://www.olx.com.pk/{cars[total_count]}-cars_c84?page={page}&filter=make_eq_{cars[total_count]}'
            return url
    else:
        return url

In [ ]:

page=1

total_count=0
cars=['suzuki','toyota', 'honda', 'daihatsu', 'nissan', 'hyundai', 'mitsubishi', 'kia', 'changan', 'faw', 'prince', 'mercedes', 'mazda',  'chevrolet', 'daewoo', 'isuzu', 'subaru', 'dfsk']
cities= ['punjab','islamabad-capital-territory','sindh','khyber-pakhtunkhwa','balochistan','azad-kashmir']
mobiles=['apple-iphone', 'samsung', 'vivo', 'oppo', 'infinix', 'xiaomi', 'tecno', 'one-plus', 'huawei', 'google', 'motorola', 'realme', 'nokia', 'lg', 'sony', 'honor', 'mobilink-jazzx', 'qmobile', 'asus']


full_loc=[]
names=[]
prices=[]
prices_numeric=[]
locale=[]
feature1=[]
feature1_numeric=[]
feature2=[]
feature3=[]

while True:
    url=f'https://www.olx.com.pk/{cities[total_count]}_g2003006/houses_c1721?page={page}'
        #   https://www.olx.com.pk/{cars[total_count]}-cars_c84?page={page}&filter=make_eq_cars-{cars[total_count]}
        #   https://www.olx.com.pk/{cities[total_count]}_g2003006/houses_c1721?page={page}
    
    url=returnLink(url,cars,total_count,page)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    html = driver.page_source
    parsed_html = BeautifulSoup(html, "html.parser")

    containers = parsed_html.find("ul", {"class" : "ba608fb8 de8df3a3"})
    #containers = parsed_html.find_all("div", {"class" : "_9bea76df"})
   
    count=0
    # driver.close()
    if(page%10==0):
        print("Link:",url)
    for item in containers:
        count+=1
        #print(containers.title)
        actual_box = item.find("div", {"class" : "a52608cc"})
        if(actual_box==None):
            continue
        else:
            """
            Prices
            """
            price=actual_box.find("div",{'class':'_1075545d _52497c97 _96d4439a'}).span.text
            price=price.replace("Rs","Rs. ")
            prices.append(price)
            
            price=price.replace("Rs. ","")
            price=price.replace(",","")
            prices_numeric.append(int(price))

            """
            Name of the Product
            """
            title=actual_box.find("div",{'class':'a5112ca8 _5fdf4379'}).p.text
            names.append(title)

            """
            Location of the Product
            """
            place=actual_box.find("div",{'class':'e48cb10f'}).span.text
            place=place.replace('•',"")
            full_loc.append(place)
            place=place.split(',')
            locale.append(place[-1])#show city

            
            """
            Car Inforamtion

            """
            if 'car' or 'house' in url:
                car_info=actual_box.find("div",{'class':'_1075545d _47c29360 _96d4439a'})
                #print(car_info)
                miles=actual_box.find("span",{'class':'e021be12'}).text
                feature1.append(miles)
                if 'car' in url:
                    miles=miles.replace("km","")
                    miles=miles.replace(",","")
                    feature1_numeric.append(int(miles))
                #print(miles)
                years=actual_box.find("span",{'class':'e05e3d9c _2f543a00'}).text
                feature2.append(years)
                if 'house' in url:
                    wash=car_info.find("span",{'aria-label':'Bathrooms'}).text
                    feature3.append(wash)
                    
    feature1 = [item.replace('+', '').replace('Studio','1') for item in feature1]
    feature3 = [item.replace('+', '') for item in feature3]
    
    if(page==1):
        if 'car' in url:
            df = pd.DataFrame(list(zip(names,prices,prices_numeric,locale,feature1,feature1_numeric,feature2)), columns =['Name','Price','Price (Value)','City','Distance','Distance (Value)','Year'])
        elif 'mobile' in url:
            df = pd.DataFrame(list(zip(names,prices,prices_numeric,locale)), columns =['Name','Price','Price (Value)','City'])
        else:
            df = pd.DataFrame(list(zip(names,prices,prices_numeric,locale,full_loc,feature1,feature2,feature3)), columns =['Name','Price','Price (Value)','City','Full Location','Bedrooms','Area','Bathroom'])

    else:
        if 'car' in url:
            df2 = pd.DataFrame(list(zip(names,prices,prices_numeric,locale,feature1,feature1_numeric,feature2)), columns =['Name','Price','Price (Value)','City','Distance','Distance (Value)','Year'])
        elif 'mobile' in url:
            df2 = pd.DataFrame(list(zip(names,prices,prices_numeric,locale)), columns =['Name','Price','Price (Value)','City'])
        else:
            df2 = pd.DataFrame(list(zip(names,prices,prices_numeric,locale,full_loc,feature1,feature2,feature3)), columns =['Name','Price','Price (Value)','City','Full Location','Bedrooms','Area','Bathroom'])
        df = pd.concat([df, df2], ignore_index = True, axis = 0)
        
    next_page=parsed_html.find("a", {"class" : "_95dae89d"})
    if(next_page):
        page+=1
        continue
    else:
        print("No more pages left\n")
        if(total_count==0):
            if 'house' not in url:
                category=getCategory(url)   
                brand=getBrand(url)
                df['Category']=category
                df['Sub-Category']=brand
                df3=df
                mask = df3['Name'].str.contains(brand, case=False) 
                df = df[mask]
            final_df=df.copy(deep=True)
        else:
            if 'house' not in url:
                print("it shouldn't be in here")
                category=getCategory(url)   
                brand=getBrand(url)
                df['Category']=category
                df['Sub-Category']=brand
                df3=df
                mask = df3['Name'].str.contains(brand, case=False) 
                df = df[mask]
            final_df = pd.concat([df, final_df], ignore_index = True, axis = 0)

        total_count+=1
        page=1
        if(total_count==len(cities)-1):
            break
        else:
            continue
    



print(count,total_count)
if os.path.exists("result.csv"):
    data2=pd.read_csv("result.csv")
    data1 = pd.concat([final_df, data2], ignore_index = True, axis = 0)
    data1.drop_duplicates(inplace=True)
    data1.to_csv(r'result.csv', index = False)   
else:
    print("Results file does not exist. Creating one")
    final_df.drop_duplicates(inplace=True)
    final_df.to_csv(r'result.csv', index = False)
    final_df.tail()


